In [1]:
import boto3

In [2]:
table_name = "car_listings"


In [5]:
client = boto3.client('dynamodb')
DB = boto3.resource('dynamodb')
table = DB.Table(table_name)

### Describe Table

In [10]:
response = client.describe_table(TableName=table_name)
# Getting number of items, could be potential ID
response['Table']['ItemCount']

0

In [21]:
import pandas as pd
from os import listdir
import re

def capitalize(make):
    
    make = re.sub("_", " ", make)
    make = make.split(" ")
    
    make = [i.capitalize() for i in make]
    
    make = " ".join(make)
    
    return make

def extract_date(name):
    return int(re.search(r"(\d{4})", name).group(1))

def clean_mileage(miles):
    
    mileage = re.sub(',', '', miles)
    mileage = re.sub(' mi.', '', mileage)
    
    return int(mileage)

def clean_rating_count(rating_count):
    
    words_to_remove = ['\(', '\)', ' reviews', ',', ' review']
    
    for i in words_to_remove:
        rating_count = re.sub(i, '', rating_count)
    
    return int(rating_count)

def clean_price(price):
    
    price = re.sub(',', '', price)
    price = price.strip('$')
    
    if price == 'Not Priced':
        return None
    
    return int(price)


# Aggregating and Cleaning the data

files = listdir('data')

data_all = []

for i in files:

    file_path = "data/{}".format(i)
    
    df = pd.read_csv(file_path, on_bad_lines = 'skip')
    df['Make'] = i[:-4]
    df = df.drop(columns=["Unnamed: 0"])
    
    data_all.append(df)
    
df_all = pd.concat(data_all, axis = 0, ignore_index=True)

df_transform = df_all.copy()
df_transform = df_transform.dropna()

df_transform['Make'] = df_all['Make'].apply(lambda x: capitalize(x))
df_transform['Name'].apply(lambda x: extract_date(x))
df_transform['Year'] = df_all['Name'].apply(lambda x: extract_date(x))
df_transform['Mileage'] = df_transform['Mileage'].apply(lambda x: clean_mileage(x))
df_transform['Rating Count'] = df_transform['Rating Count'].apply(lambda x: clean_rating_count(x))
df_transform['Price'] = df_transform['Price'].apply(lambda x: clean_price(x))

df_transform = df_transform.dropna()

acura.csv
alfa_romeo.csv
aston_martin.csv
bmw.csv
buick.csv
cadillac.csv
chevrolet.csv
chrysler.csv
fiat.csv
ford.csv
gmc.csv
honda.csv
infiniti.csv
jeep.csv
kia.csv
maserati.csv
mercedes_benz.csv
mini.csv
mitsubishi.csv
nissan.csv
porsche.csv
ram.csv
rolls_royce.csv
subaru.csv
toyota.csv
volkswagen.csv
volvo.csv


In [22]:
df_transform.head()

,Name,Mileage,Dealer Name,Rating,Rating Count,Price,Make,Year
0,2020 Acura TLX Technology,13214,Piazza Acura of Ardmore,4.6,138,31999.0,Acura,2020
1,2020 Acura TLX FWD,35713,Acura Carland,4.5,145,33989.0,Acura,2020
2,2020 Acura MDX 3.5L,23967,Pohanka Acura,4.8,1662,39128.0,Acura,2020
3,2020 Acura MDX 3.5L,16437,Hiley Acura,3.7,61,41689.0,Acura,2020
4,2019 Acura MDX 3.5L,33408,Jay Wolfe Acura of Overland Park,4.8,1498,38051.0,Acura,2019


In [29]:
item = df_transform.iloc[0].to_dict()
item['Date']

{'Name': '2020 Acura TLX Technology', 'Mileage': 13214, 'Dealer Name': 'Piazza Acura of Ardmore', 'Rating': 4.6, 'Rating Count': 138, 'Price': 31999.0, 'Make': 'Acura', 'Year': 2020}


In [35]:
from datetime import datetime
today = datetime.now().date().isoformat()
today

'2022-08-06'

In [ ]:
primary_key = 0 

with table.batch_writer() as batch:
    
    